In [47]:
import glob
import numpy as np

# extract job location
folder_to_fix = '/mnt/HDD/CS-hdd-gpu/J1467/'

files = np.array(sorted(glob.glob(folder_to_fix + '/*.cs')))
for n, file in enumerate(files):
    print(f'{n} {file}')

0 /mnt/HDD/Dawid_CS/CS-measles-hdd-gpu/J1467/J1467_passthrough_micrographs.cs
1 /mnt/HDD/Dawid_CS/CS-measles-hdd-gpu/J1467/J1467_passthrough_micrographs_incomplete.cs
2 /mnt/HDD/Dawid_CS/CS-measles-hdd-gpu/J1467/J1467_passthrough_particles.cs
3 /mnt/HDD/Dawid_CS/CS-measles-hdd-gpu/J1467/extracted_particles.cs
4 /mnt/HDD/Dawid_CS/CS-measles-hdd-gpu/J1467/incomplete_micrographs.cs
5 /mnt/HDD/Dawid_CS/CS-measles-hdd-gpu/J1467/picked_micrographs.cs


In [48]:
# select here extracted particles and passthrough particles files
files_to_fix = files[[2,3]]

for n, file in enumerate(files_to_fix):
    print(f'{n} {file}')
    
# make sure only those to are listed

0 /mnt/HDD/Dawid_CS/CS-measles-hdd-gpu/J1467/J1467_passthrough_particles.cs
1 /mnt/HDD/Dawid_CS/CS-measles-hdd-gpu/J1467/extracted_particles.cs


In [57]:
# those need to identified inside cryosparc job shown in the error message
broken_path = [b'J1467/extract/FoilHole_22118001_Data_22118427_22118429_20240422_170657_fractions_patch_aligned_doseweighted_particles.mrc',
               b'J1467/extract/FoilHole_22118002_Data_22118430_22118432_20240422_170834_fractions_patch_aligned_doseweighted_particles.mrc',
               b'J1467/extract/FoilHole_22118025_Data_22118433_22118435_20240422_173624_fractions_patch_aligned_doseweighted_particles.mrc',
               b'J1467/extract/FoilHole_22118044_Data_22118445_22118447_20240422_180307_fractions_patch_aligned_doseweighted_particles.mrc',
               b'J1467/extract/FoilHole_22118001_Data_22118430_22118432_20240422_170705_fractions_patch_aligned_doseweighted_particles.mrc',
               b'J1467/extract/FoilHole_22118029_Data_22118433_22118435_20240422_174116_fractions_patch_aligned_doseweighted_particles.mrc',
               b'J1467/extract/FoilHole_22118018_Data_22118442_22118444_20240422_172444_fractions_patch_aligned_doseweighted_particles.mrc',
               b'J1467/extract/FoilHole_22118003_Data_22118427_22118429_20240422_170958_fractions_patch_aligned_doseweighted_particles.mrc',
               b'J1467/extract/FoilHole_22118031_Data_22118430_22118432_20240422_174421_fractions_patch_aligned_doseweighted_particles.mrc']

for file in files_to_fix:
    file = np.load(file, allow_pickle=False)
    print(file.dtype)
    try:
        to_remove = np.where(np.isin(file['blob/path'], broken_path))[0]
        print(to_remove)
    except:
        pass

[('uid', '<u8'), ('pick_stats/ncc_score', '<f4'), ('pick_stats/power', '<f4'), ('pick_stats/template_idx', '<u4'), ('pick_stats/angle_rad', '<f4'), ('ml_properties/type', 'S6')]
[('uid', '<u8'), ('blob/path', 'S122'), ('blob/idx', '<u4'), ('blob/shape', '<u4', (2,)), ('blob/psize_A', '<f4'), ('blob/sign', '<f4'), ('blob/import_sig', '<u8'), ('location/micrograph_uid', '<u8'), ('location/exp_group_id', '<u4'), ('location/micrograph_path', 'S118'), ('location/micrograph_shape', '<u4', (2,)), ('location/micrograph_psize_A', '<f4'), ('location/center_x_frac', '<f4'), ('location/center_y_frac', '<f4'), ('ctf/type', 'S7'), ('ctf/exp_group_id', '<u4'), ('ctf/accel_kv', '<f4'), ('ctf/cs_mm', '<f4'), ('ctf/amp_contrast', '<f4'), ('ctf/df1_A', '<f4'), ('ctf/df2_A', '<f4'), ('ctf/df_angle_rad', '<f4'), ('ctf/phase_shift_rad', '<f4'), ('ctf/scale', '<f4'), ('ctf/scale_const', '<f4'), ('ctf/shift_A', '<f4', (2,)), ('ctf/tilt_A', '<f4', (2,)), ('ctf/trefoil_A', '<f4', (2,)), ('ctf/tetra_A', '<f4', (

In [58]:
import shutil
import time

for file_path in files_to_fix:
        # Load the numpy array
        data = np.load(file_path, allow_pickle=False)
        
        # Assuming to_remove is an array of indices, use np.delete to remove rows
        modified_data = np.delete(data, to_remove, axis=0)
        
        # Backup the original file
        backup_path = file_path + f'.bak{int(time.time())}'
        shutil.move(file_path, backup_path)
        
        # Save the modified array, defining dtypes if needed
        np.save(file_path, modified_data, allow_pickle=False)
        shutil.move(file_path+'.npy', file_path)   